In [1]:
from selenium.webdriver import Chrome
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from datetime import datetime, timedelta
import pandas as pd
import time
import openpyxl

In [2]:
browser = Chrome()

browser.get('https://store.musinsa.com/') #무신사 스토어

# browser.find_element_by_css_selector('span#search_button').click()
search_keyword = browser.find_element_by_css_selector('input#search_query')
search_keyword.click()
search_keyword.send_keys('무신사 스탠다드')
search_keyword.submit()

browser.find_element_by_css_selector('a.brand-shop__link').click()

categories = browser.find_elements_by_css_selector('li.categoty_tab a.title')
for category in categories:
    if '후기' in category.text:
        category.click()

In [3]:
# xlsx파일
# 뉴스 내용에 ','가 들어가 있는 경우, csv 파일로는 정상적으로 저장할 수 없다고 판단 -> xlsx 파일로 저장
wb = openpyxl.Workbook()
sheet = wb.active
sheet.append('이름,성별,별점,내용'.split(','))

myDate = pd.to_datetime(datetime.today() - timedelta(days = 365)) # 1년 전 리뷰까지 가져옴.
while True:
    pages = browser.find_elements_by_css_selector('.sorter-box a.paging-btn')
    page_num = [page.get_attribute('data-page') for page in pages]
    for i in range(1,len(page_num)):
        browser.execute_script("window.scrollTo(0, 500)")
        WebDriverWait(browser, 20).until(EC.element_to_be_clickable((By.CSS_SELECTOR, f".sorter-box a.paging-btn[data-page^='{page_num[i]}']"))).click()

        if i == len(page_num)-1:
            continue
            
#         print('page='+page_num[i])
        
        reviews = browser.find_elements_by_css_selector('div.postContents')
        for review in reviews:
            try:
                name = review.find_element_by_css_selector('.list_info.p_name').text
            except:
                name = '답변없음'
            try:
                gender = review.find_element_by_css_selector('.estimate-profile').text.split(',')[0]
            except:
                gender = '답변없음'
            content = review.find_element_by_css_selector('div.summary span.content-review').text
            try:
                review.find_element_by_css_selector('span.score.score10')
                star = 5
            except:
                try:
                    review.find_element_by_css_selector('span.score.score8')
                    star = 4
                except:
                    try:
                        review.find_element_by_css_selector('span.score.score6')
                        star = 3
                    except:
                        try:
                            review.find_element_by_css_selector('span.score.score4')
                            star = 2
                        except:
                            try:
                                review.find_element_by_css_selector('span.score.score2')
                                star = 1
                            except:
                                star = 0
                                
            try:
                date = pd.to_datetime(review.find_element_by_css_selector('div.profile span.date').text,format='%y.%m.%d')
            except:
                date = pd.to_datetime(datetime.today())
            
            try:
                sheet.append([i for i in [name, gender, star, content]])
            except:
                continue
    
    if date <= myDate:
        break
        
wb.save('Musinsa_standard.xlsx')
browser.close()